In [1]:
import os
import point_cloud_playground
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Load Data

In [2]:
data_folder = './data'

In [3]:
files = os.listdir(data_folder)

In [4]:
point_cloud_files = [file for file in files if file.endswith(('.ply'))]

In [5]:
point_clouds = []
for file in point_cloud_files:
    file_path = os.path.join(data_folder,file)
    point_cloud = o3d.io.read_point_cloud(file_path)
    point_clouds.append((point_cloud,file))

### visualize data

In [6]:
o3d.visualization.draw_geometries(
    [point_clouds[0][0]],
    window_name="Open3D Visualization",
    width=800,
    height=600,
    left=50,
    top=50,
    point_show_normal=True
)

In [7]:
import numpy as np
np.asarray(point_cloud.points)

array([[-0.1012109 ,  0.07229348,  0.7001285 ],
       [-0.07348169,  0.08817801,  0.71163565],
       [-0.01457525,  0.08745149,  0.70577228],
       ...,
       [ 0.15158254, -0.48567602,  1.40445578],
       [ 0.11910991, -0.48977143,  1.43934286],
       [ 0.10471008, -0.49163854,  1.45536315]])

## FLoor Detection 

In [8]:

from point_cloud_playground import utils
def detect_and_visualize(point_cloud,PlaneDetector):
    pc = point_cloud[0]
    pc_name = point_cloud[1]
    image_file_name = "./images/"+str(PlaneDetector.__name__)+"_"+str(pc_name)+".png"
    plane_detector = PlaneDetector(pc)
    floor_equation = plane_detector.detect_plane()
    floor_mesh = utils.create_plane_from_equation(floor_equation,plane_size=1)
    try:
        visualization_window = o3d.visualization.draw_geometries(
            [floor_mesh, plane_detector.point_cloud],
            window_name=str(PlaneDetector.__name__)+"_"+str(pc_name),
            width=800,
            height=600,
            left=50,
            top=50,
            point_show_normal=True
        )
        if visualization_window is not None:
            visualization_window.capture_screen_image(image_file_name)
            print("Image saved as:", image_file_name)
        else:
            print("Error: Visualization window is None.")
    except Exception as e:
        print("Error:", e)

In [9]:
from point_cloud_playground.plane_detector import PlaneDetectorRANSAC, PlaneDetectorConvexHull, PlaneDetectorPCA

for pc in point_clouds:
    detect_and_visualize(pc,PlaneDetectorRANSAC)


Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.
Error: Visualization window is None.


## FLoor Reorientation

In [10]:
plane_detector = PlaneDetectorRANSAC(point_clouds[1][0])
plane_detector.detect_plane()
status = plane_detector.reorient_plane()
new_floor_equation = plane_detector.detect_plane()

In [11]:
o3d.visualization.draw_geometries(
    [plane_detector.point_cloud],
    window_name="Open3D Visualization",
    width=800,
    height=600,
    left=50,
    top=50,
    point_show_normal=True
)

In [12]:
new_floor_equation

(0.9999959509295026,
 0.0028038956794303417,
 0.0004861004204195983,
 424.61832078134086)

## Floor Smoother

In [13]:
from point_cloud_playground.point_cloud_smoother import PointCloudSmoother
smoother = PointCloudSmoother(plane_detector.point_cloud)
smooth_mesh = smoother.poisson_surface_reconstruction()

In [14]:
o3d.visualization.draw_geometries(
    [smooth_mesh],
    window_name="Smooth Mesh",
    width=800,
    height=600,
    left=50,
    top=50,
    point_show_normal=True
)